# Intro to Discrete Events Simulation with Python

<!-- [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/vitostamatti/discrete-event-simulation-simpy/blob/main/notebooks/basic_examples.ipynb) -->

In this set of examples, I'll explore the `simpy` library and it's main
objects. It's not the most intuitive library but not because of the design
or quality of it, but for the non trivial problem that it aims to solve.

Discrete Event Simulation with python is not an easy task, and I'm far from
being an expert in this matter, so I apologies in advance if a make a wrong
use of the simpy library or the concepts behind it.


## 1. Hello world example

First example is just to ilustrate how to start a simulation and what simpy needs in order to make its magic.

First of all, we need to import and install simpy. Then, to make an event ocurr we need to use python generators. The explanation of generators if far beyond this notebook so I strongly recommend to take a look on to the [official python documentation](https://docs.python.org/3/reference/expressions.html#yieldexpr)

The way I like to think about generators in this specific scenario is that they will "freeze" the excecution until a condition is passed. In our case, this conditions is the env.timeout() event, which internally makes the simulation time to advance. After this timeout is finished, the excecution continues from where it was interrupted.


In [12]:
import simpy


def example(env):
    # we are about to freeze excecution for 1 unit
    print(f"Before timeout: now={env.now}")
    value = yield env.timeout(1, value=42)
    # now we returned to the excecution and continue
    print(f"After timeout: now={env.now}, value={value}")


env = simpy.Environment()
p = env.process(example(env))
env.run()

Before timeout: now=0
After timeout: now=1, value=42


An alternative to python functions, is to use python classes (which I personally prefer).
This allows us to build much more "interpretable" models where the simulation objects
represents real objects.


In [14]:
class Example:
    def __init__(self, env, delay=10):
        self.env = env
        self.delay = delay

    def process(self):
        # we are about to freeze excecution for 1 unit
        print(f"Before timeout: now={self.env.now}")
        value = yield self.env.timeout(1, value=42)
        # now we returned to the excecution and continue
        print(f"After timeout: now={self.env.now}, value={value}")


env = simpy.Environment()
e = Example(env, delay=10)
env.process(e.process())
env.run()

Before timeout: now=0
After timeout: now=1, value=42


## 2. Vehicle example

In this example, we'll make a process to run uninterruptedly
and to let simpy enviroment finish the simulation when a maximum time
is reached. This kind of simulation it's mostly use to emulate
real time dependent process and to evaluate it's progress over time.


In [18]:
import simpy
import numpy as np


class VehicleExample:
    """
    Vehicle object that can drive() for a given time
    depending on its fuel consumption and fuel capacity.

    Attributes:
        env (simpy.Environment): the simpy environment to run with.
        min_speed (float, optional): min speed expressed in km/hs. Defaults to 80.
        max_speed (float, optional): max speed expressed in km/hs. Defaults to 100.
        fuel_consumption (float, optional): fuel consumption expressed in km/lt. Defaults to 10.
        fuel_capacity (float, optional): fuel capacity expressed in lt. Defaults to 35.
    """

    def __init__(
        self,
        env: simpy.Environment,
        min_speed: float = 80.0,
        max_speed: float = 100,
        fuel_consumption: float = 10,
        fuel_capacity: float = 35,
    ):
        self.env = env
        self.min_speed = min_speed  # km/hs
        self.max_speed = max_speed  # km/hs
        self.fuel_consumption = fuel_consumption  # km/lt
        self.fuel_capacity = fuel_capacity  # lt
        self.fuel = fuel_capacity  # lt

    def drive(self):
        """Excecutes the loop that makes the vehicle drive"""
        while True:
            print(f"Start Driving at {self.env.now:.2f}")
            travel_time = (self.fuel * self.fuel_consumption) / (
                np.random.randint(self.min_speed, self.max_speed)
            )
            yield self.env.timeout(travel_time)
            print(f"Need refueling at {self.env.now:.2f}")

            print(f"Start refueling at {self.env.now:.2f}")
            yield self.env.timeout(np.random.uniform(0.05, 0.15))
            self.fuel = np.random.randint(self.fuel_capacity - 5, self.fuel_capacity)
            print(
                f"Finished refueling at {self.env.now:.2f}: fuel now is {self.fuel:.2f}"
            )


env = simpy.Environment()
v = VehicleExample(env)
env.process(v.drive())
env.run(until=24)

Start Driving at 0.00
Need refueling at 3.98
Start refueling at 3.98
Finished refueling at 4.09: fuel now is 33.00
Start Driving at 4.09
Need refueling at 8.12
Start refueling at 8.12
Finished refueling at 8.24: fuel now is 32.00
Start Driving at 8.24
Need refueling at 12.00
Start refueling at 12.00
Finished refueling at 12.12: fuel now is 33.00
Start Driving at 12.12
Need refueling at 16.19
Start refueling at 16.19
Finished refueling at 16.28: fuel now is 30.00
Start Driving at 16.28
Need refueling at 19.44
Start refueling at 19.44
Finished refueling at 19.55: fuel now is 34.00
Start Driving at 19.55
Need refueling at 23.17
Start refueling at 23.17
Finished refueling at 23.29: fuel now is 30.00
Start Driving at 23.29


## 3. Machine Example

In this example, we simulate a machine that processes products in
a queue or production plan. The policy used to select the next product
can be:

- Random
- FIFO (first in, first out)
- LIFO (last in, first out)


In [10]:
import copy


class Product:
    def __init__(self, id):
        self.id = id
        self.processed = False


class Machine:
    def __init__(self, env: simpy.Environment, id, process_time=(10, 15, 20)):
        self.env = env
        self.id = id
        self.process_time = process_time
        self.idle = True

        self.pending_products = []
        self.processed_products = []

    def generate_process_time(self):
        return np.random.triangular(
            self.process_time[0], self.process_time[1], self.process_time[2]
        )

    def process_product(self, product):
        print(f"Processing product {product.id} at time {self.env.now}")
        self.idle = False
        yield self.env.timeout(delay=self.generate_process_time())
        product.processed = True
        self.idle = False
        return product

    def _set_production_plan(self, products):
        self.pending_products = copy.deepcopy(products)
        self.processed_products = []

    def process_all_products_randomly(self, products):
        self._set_production_plan(products)
        while len(self.pending_products) > 0:
            next_prod = np.random.choice(range(len(self.pending_products)))
            p = self.pending_products[next_prod]
            yield env.process(m.process_product(p))
            self.pending_products.remove(p)
            self.processed_products.append(p)

    def process_all_products_fifo(self, products):
        self._set_production_plan(products)
        while len(self.pending_products) > 0:
            p = self.pending_products[0]
            yield env.process(m.process_product(p))
            self.pending_products.remove(p)
            self.processed_products.append(p)

    def process_all_products_lifo(self, products):
        self._set_production_plan(products)
        while len(self.pending_products) > 0:
            p = self.pending_products[-1]
            yield env.process(m.process_product(p))
            self.pending_products.remove(p)
            self.processed_products.append(p)


print("Processing RANDOM policy")
env = simpy.Environment()
m = Machine(env, id="machine")
products = [Product(id=i) for i in range(5)]
env.process(m.process_all_products_randomly(products))
env.run()
print()

print("Processing FIFO policy")
env = simpy.Environment()
m = Machine(env, id="machine")
products = [Product(id=i) for i in range(5)]
env.process(m.process_all_products_fifo(products))
env.run()
print()

print("Processing LIFO policy")
env = simpy.Environment()
m = Machine(env, id="machine")
products = [Product(id=i) for i in range(5)]
env.process(m.process_all_products_lifo(products))
env.run()

Processing RANDOM policy
Processing product 0 at time 0
Processing product 3 at time 14.664437457331555
Processing product 4 at time 28.43934381274208
Processing product 2 at time 40.93311214678734
Processing product 1 at time 54.521131103061315

Processing FIFO policy
Processing product 0 at time 0
Processing product 1 at time 16.6348964608339
Processing product 2 at time 34.13186488794743
Processing product 3 at time 49.710138733369114
Processing product 4 at time 63.841699204302074

Processing LIFO policy
Processing product 4 at time 0
Processing product 3 at time 16.84107861088087
Processing product 2 at time 30.824374256889396
Processing product 1 at time 46.09430196538585
Processing product 0 at time 60.60309375245397
